# Generating molecular descriptors and fingerprints to predict oral bioavailability

1. This notebook focuses on generating molecular descriptors/fingerprints and using them to build a random forest classifier to predict classes of oral bioavailability.
2. Molecular descriptors/fingerprints were generated using RDkit and random forest classifer model was built using sci-kit learn. 
3. Hyperparameters tuning was done using Optuna library in 30 trials.

In [33]:
import rdkit
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from rdkit import Chem
import pandas as pd
import numpy as np
from numpy import mean, std
from rdkit.ML.Descriptors import MoleculeDescriptors
import optuna
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score

from config import MINPATH, MAXPATH, FPSIZE_RDKFP, MORGAN_RAD, MORGAN_BITS, N_SPLITS, SEED_NO, best_params_MD, best_params_morgan, best_params_maccskeys, best_params_rdkfp
from utils import seed_everything

## 1. Process data and store them as csv files

In [2]:
def generate_mol_desc(dataset_train_path, dataset_test_path, saved_path_train, saved_path_test):
    df = pd.read_csv(dataset_train_path)
    df_test = pd.read_csv(dataset_test_path)
    smiles_df = df['smile']
    smiles_test_df = df_test['smile']
    all_descs =[x[0] for x in Descriptors._descList]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(all_descs)

    desc_list = []
    for smile in smiles_df:
        mol = Chem.MolFromSmiles(smile)
        desc = calc.CalcDescriptors(mol)
        desc_list.append(desc)
    df_desc = pd.DataFrame(desc_list, columns=all_descs)

    desc_list_test = []
    for smile in smiles_test_df:
        mol = Chem.MolFromSmiles(smile)
        desc = calc.CalcDescriptors(mol)
        desc_list_test.append(desc)
    df_desc_test = pd.DataFrame(desc_list_test, columns=all_descs)


    col_with_all_zeros = []
    for i in df_desc.columns:
        if (df_desc[i]== 0).all():
            col_with_all_zeros.append(i)

    df_desc = df_desc.drop(col_with_all_zeros, axis=1)
    df_desc_test = df_desc_test.drop(col_with_all_zeros, axis=1)

    data = pd.DataFrame(df_desc.var(), columns=['variance']).T

    col_list_zero_var = []
    for i in data.columns:
        if (df_desc[i] == 0.0).any():
            col_list_zero_var.append(i)

    df_desc = df_desc.drop(col_list_zero_var, axis=1)
    df_desc_test = df_desc_test.drop(col_list_zero_var, axis=1)

    df_desc['label_cutoff_50%'] = df['label_cutoff_50%']
    df_desc['smile'] = df['smile']
    df_desc_test['label_cutoff_50%'] = df_test['label_cutoff_50%']
    df_desc_test['smile'] = df_test['smile']

    df_desc.to_csv(saved_path_train, index=False)
    df_desc_test.to_csv(saved_path_test, index=False)
    print('Data processed!')

def get_maccskeys(mol):
    return np.array(MACCSkeys.GenMACCSKeys(mol))

def get_rdkfp(mol, minpath, maxpath, fpsize):
    rdkfp = Chem.rdmolops.RDKFingerprint(mol, minPath=minpath, maxPath=maxpath, fpSize = fpsize)
    return np.array(rdkfp)

def get_mfp(mol,rad, bits):
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol,radius=rad, nBits=bits)
    return np.array(morgan_fp)

def get_fp(type, dataset_train_path, dataset_test_path, saved_path_train, saved_path_test):
    df = pd.read_csv(dataset_train_path)
    df_test = pd.read_csv(dataset_test_path)
    smiles_df = df['smile']
    smiles_test_df = df_test['smile']

    fp_list_train=[]
    fp_list_test=[]

    for smile in smiles_df:
        mol = Chem.MolFromSmiles(smile)
        if type =="MACCSkeys":
            fp = get_maccskeys(mol)
        elif type =="RDKFP":
            fp = get_rdkfp(mol, MINPATH, MAXPATH, FPSIZE_RDKFP)
        elif type =="Morgan":
            fp = get_mfp(mol, MORGAN_RAD, MORGAN_BITS)
        fp_list_train.append(fp)
    
    for smile in smiles_test_df:
        mol = Chem.MolFromSmiles(smile)
        if type =="MACCSkeys":
            fp = get_maccskeys(mol)
        elif type =="RDKFP":
            fp = get_rdkfp(mol, MINPATH, MAXPATH, FPSIZE_RDKFP)
        elif type =="Morgan":
            fp = get_mfp(mol, MORGAN_RAD, MORGAN_BITS)
        fp_list_test.append(fp)
    
    train_df = pd.DataFrame(fp_list_train)
    train_df['label_cutoff_50%'] = df['label_cutoff_50%']
    train_df['smile'] = df['smile']
    train_df.to_csv(saved_path_train, index=False)
    test_df = pd.DataFrame(fp_list_test)
    test_df['label_cutoff_50%'] = df_test['label_cutoff_50%']
    test_df['smile'] = df_test['smile']
    test_df.to_csv(saved_path_test, index=False)
    print("Data processed!")

In [6]:
# 1. Get Molecular descriptors, remove columns where all values are 0 and zero variance and save as csv file 
dataset_train_path = './data/graph_data/data_oral_avail_train/raw/data_oral_avail_train_50.csv'
dataset_test_path = './data/graph_data/data_oral_avail_test/raw/data_oral_avail_test_1_50.csv'
saved_path_train = './data/oral_avail_mol_desc/data_oral_avail_train_mol_desc.csv'
saved_path_test = './data/oral_avail_mol_desc/data_oral_avail_test_mol_desc.csv'
generate_mol_desc(dataset_train_path, dataset_test_path, saved_path_train, saved_path_test)

Data processed!


In [7]:
# 2. Get Morgan fingerprints

type = 'Morgan'
dataset_train_path = './data/graph_data/data_oral_avail_train/raw/data_oral_avail_train_50.csv'
dataset_test_path = './data/graph_data/data_oral_avail_test/raw/data_oral_avail_test_1_50.csv'
saved_path_train = './data/oral_avail_fingerprints/data_oral_avail_train_morgan.csv'
saved_path_test = './data/oral_avail_fingerprints/data_oral_avail_test_morgan.csv'
get_fp(type, dataset_train_path, dataset_test_path, saved_path_train, saved_path_test)


Data processed!


In [8]:
# 3. Get rdk fingerprints

type = 'RDKFP'
dataset_train_path = './data/graph_data/data_oral_avail_train/raw/data_oral_avail_train_50.csv'
dataset_test_path = './data/graph_data/data_oral_avail_test/raw/data_oral_avail_test_1_50.csv'
saved_path_train = './data/oral_avail_fingerprints/data_oral_avail_train_rdkfp.csv'
saved_path_test = './data/oral_avail_fingerprints/data_oral_avail_test_rdkfp.csv'
get_fp(type, dataset_train_path, dataset_test_path, saved_path_train, saved_path_test)


Data processed!


In [3]:
# 4. Get maccskeys

type = 'MACCSkeys'
dataset_train_path = './data/graph_data/data_oral_avail_train/raw/data_oral_avail_train_50.csv'
dataset_test_path = './data/graph_data/data_oral_avail_test/raw/data_oral_avail_test_1_50.csv'
saved_path_train = './data/oral_avail_fingerprints/data_oral_avail_train_maccskeys.csv'
saved_path_test = './data/oral_avail_fingerprints/data_oral_avail_test_maccskeys.csv'
get_fp(type, dataset_train_path, dataset_test_path, saved_path_train, saved_path_test)


Data processed!


## 2. Tuning of model (Suggest to skip and used the hyperparameters saved in config.py)
1. To tune the model, we make use of the Optuna library which allows us to use the Tree-structure Parzen Estimator Algorithm to find the best hyperparameters for us 
2. First, create a run_tuning function to include the training and validation step with early stopping mechansim 
3. Then, create an objective function of Optuna to evaluate the best hyperparameters in 30 trials. 

In [30]:
def run_tuning(data_type, processed_train_path, params):
    seed_everything(SEED_NO)
    df = pd.read_csv(processed_train_path)
    if data_type == "MD":
        x_train = df.iloc[:, :45]
        y_train = df['label_cutoff_50%']
    if data_type == "MACCSkeys":
        x_train = df.iloc[:, :167]
        y_train = df['label_cutoff_50%']
    if data_type == "Morgan":
        x_train = df.iloc[:, :1024]
        y_train = df['label_cutoff_50%']
    if data_type == "rdkfp":
        x_train = df.iloc[:, :1024]
        y_train = df['label_cutoff_50%']

    kf = KFold(n_splits=5)

    rf_model = RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'])
    logloss_total = 0

    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(x_train)):
        rf_model.fit(x_train.iloc[train_idx, :], y_train.iloc[train_idx].to_numpy().reshape(-1))
        y_pred = rf_model.predict(x_train.iloc[valid_idx, :])
        probs = rf_model.predict_proba(x_train.iloc[valid_idx, :])

        logloss = log_loss(y_train.iloc[valid_idx], probs)
        logloss_total += logloss
        print(f'Fold: {fold_no}, logloss: {logloss}')
    
    return logloss_total/5

In [31]:
# For mol desc

data_type = 'MD'
processed_train_path = './data/oral_avail_mol_desc/data_oral_avail_train_mol_desc.csv'

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 10, 100)
    }

    loss = run_tuning(data_type, processed_train_path, params)
    return loss
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print(f'best trial:')
trial_ = study.best_trial
print(trial_.values)
print(f'Best parameters: {trial_.params}')

[I 2023-03-30 22:18:48,909] A new study created in memory with name: no-name-fcd5f805-6abc-4d48-8c7b-7151d29934a9


Fold: 0, logloss: 0.5641162776598053
Fold: 1, logloss: 0.590122725650923
Fold: 2, logloss: 0.6853439128203132
Fold: 3, logloss: 0.6733378917487506


[I 2023-03-30 22:18:50,015] Trial 0 finished with value: 0.6334135586186024 and parameters: {'n_estimators': 57, 'max_depth': 92}. Best is trial 0 with value: 0.6334135586186024.


Fold: 4, logloss: 0.6541469852132198
Fold: 0, logloss: 0.5836731014212858
Fold: 1, logloss: 0.6308441055034912
Fold: 2, logloss: 0.6817913712281339


[I 2023-03-30 22:18:50,867] Trial 1 finished with value: 0.6460592791458533 and parameters: {'n_estimators': 44, 'max_depth': 14}. Best is trial 0 with value: 0.6334135586186024.


...

Fold: 4, logloss: 0.6640685735459151
Fold: 0, logloss: 0.5652616637228591
Fold: 1, logloss: 0.6237008885226218
Fold: 2, logloss: 0.6779326440151046
Fold: 3, logloss: 0.6776538092025096


[I 2023-03-30 22:19:24,240] Trial 28 finished with value: 0.6416023242885158 and parameters: {'n_estimators': 64, 'max_depth': 81}. Best is trial 2 with value: 0.6331825550307445.


Fold: 4, logloss: 0.6634626159794837
Fold: 0, logloss: 0.5878153835110843
Fold: 1, logloss: 0.6293873649480071
Fold: 2, logloss: 0.7134797207552535
Fold: 3, logloss: 0.6734874396832967


[I 2023-03-30 22:19:25,260] Trial 29 finished with value: 0.6523777688340022 and parameters: {'n_estimators': 53, 'max_depth': 24}. Best is trial 2 with value: 0.6331825550307445.


Fold: 4, logloss: 0.6577189352723694
best trial:
[0.6331825550307445]
Best parameters: {'n_estimators': 76, 'max_depth': 96}


In [21]:
# For morgan

data_type = 'Morgan'
processed_train_path = './data/oral_avail_fingerprints/data_oral_avail_train_morgan.csv'

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 10, 100)
    }

    loss = run_tuning(data_type, processed_train_path, params)
    return loss
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print(f'best trial:')
trial_ = study.best_trial
print(trial_.values)
print(f'Best parameters: {trial_.params}')

[I 2023-03-30 22:10:04,802] A new study created in memory with name: no-name-3a1e6f95-359b-41dd-a970-b4aa2f25ce64


Fold: 0, logloss: 0.6037030741479498
Fold: 1, logloss: 0.6760532480779402
Fold: 2, logloss: 0.6843469733881837
Fold: 3, logloss: 0.679947487601778


[I 2023-03-30 22:10:05,392] Trial 0 finished with value: 0.6627167233682746 and parameters: {'n_estimators': 44, 'max_depth': 26}. Best is trial 0 with value: 0.6627167233682746.


Fold: 4, logloss: 0.6695328336255214
Fold: 0, logloss: 0.6095791188935008
Fold: 1, logloss: 0.6803859753252263
Fold: 2, logloss: 0.6783940435609934
Fold: 3, logloss: 0.6992144496911429


[I 2023-03-30 22:10:05,939] Trial 1 finished with value: 0.6674195788071865 and parameters: {'n_estimators': 41, 'max_depth': 22}. Best is trial 0 with value: 0.6627167233682746.


...

Fold: 4, logloss: 0.6730006463655295
Fold: 0, logloss: 0.6151357358496423
Fold: 1, logloss: 0.6557617946426755
Fold: 2, logloss: 0.6629114724287103
Fold: 3, logloss: 0.684641685841066


[I 2023-03-30 22:10:32,182] Trial 28 finished with value: 0.6556050724237645 and parameters: {'n_estimators': 92, 'max_depth': 35}. Best is trial 19 with value: 0.6503523008151717.


Fold: 4, logloss: 0.6595746733567285
Fold: 0, logloss: 0.6116322843158437
Fold: 1, logloss: 0.6588666351502431
Fold: 2, logloss: 0.6751942110708437


[I 2023-03-30 22:10:32,964] Trial 29 finished with value: 0.6610879256796578 and parameters: {'n_estimators': 63, 'max_depth': 29}. Best is trial 19 with value: 0.6503523008151717.


Fold: 3, logloss: 0.6844992863354469
Fold: 4, logloss: 0.6752472115259116
best trial:
[0.6503523008151717]
Best parameters: {'n_estimators': 89, 'max_depth': 40}


In [22]:
# For maccskeys

data_type = 'MACCSkeys'
processed_train_path = './data/oral_avail_fingerprints/data_oral_avail_train_maccskeys.csv'

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 10, 100)
    }

    loss = run_tuning(data_type, processed_train_path, params)
    return loss
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print(f'best trial:')
trial_ = study.best_trial
print(trial_.values)
print(f'Best parameters: {trial_.params}')

[I 2023-03-30 22:10:33,028] A new study created in memory with name: no-name-b065569f-3141-488e-bc08-accfd72158a5


Fold: 0, logloss: 0.5900430614725523
Fold: 1, logloss: 0.6311049664438453
Fold: 2, logloss: 0.6730864359656435


[I 2023-03-30 22:10:33,387] Trial 0 finished with value: 0.6720306416981096 and parameters: {'n_estimators': 43, 'max_depth': 58}. Best is trial 0 with value: 0.6720306416981096.


Fold: 3, logloss: 0.6940077219199089
Fold: 4, logloss: 0.7719110226885979
Fold: 0, logloss: 0.5938439551628578
Fold: 1, logloss: 0.6299928486778049
Fold: 2, logloss: 0.6676151291681576
Fold: 3, logloss: 0.682095131321041


[I 2023-03-30 22:10:33,932] Trial 1 finished with value: 0.6423073948898729 and parameters: {'n_estimators': 72, 'max_depth': 58}. Best is trial 1 with value: 0.6423073948898729.


...

Fold: 3, logloss: 0.6809748888090598
Fold: 4, logloss: 0.6391527479630589
Fold: 0, logloss: 0.6145229743239571
Fold: 1, logloss: 0.6383507813040878
Fold: 2, logloss: 0.6502859534622137


[I 2023-03-30 22:10:49,008] Trial 28 finished with value: 0.6428001588889172 and parameters: {'n_estimators': 63, 'max_depth': 71}. Best is trial 15 with value: 0.6334365205536097.


Fold: 3, logloss: 0.6672878215760952
Fold: 4, logloss: 0.6435532637782319
Fold: 0, logloss: 0.593371377200313
Fold: 1, logloss: 0.644917484370817
Fold: 2, logloss: 0.6924579881400728
Fold: 3, logloss: 0.6891860924331101


[I 2023-03-30 22:10:49,427] Trial 29 finished with value: 0.6536798553012768 and parameters: {'n_estimators': 51, 'max_depth': 63}. Best is trial 15 with value: 0.6334365205536097.


Fold: 4, logloss: 0.6484663343620714
best trial:
[0.6334365205536097]
Best parameters: {'n_estimators': 85, 'max_depth': 46}


In [24]:
# For rdkfp

data_type = 'rdkfp'
processed_train_path = './data/oral_avail_fingerprints/data_oral_avail_train_rdkfp.csv'

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 10, 100)
    }

    loss = run_tuning(data_type, processed_train_path, params)
    return loss
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print(f'best trial:')
trial_ = study.best_trial
print(trial_.values)
print(f'Best parameters: {trial_.params}')

[I 2023-03-30 22:11:14,477] A new study created in memory with name: no-name-eb0f18b4-cffd-42f5-8fab-f0d1a25daff6


Fold: 0, logloss: 0.7658011993817626
Fold: 1, logloss: 1.0278093757548183
Fold: 2, logloss: 0.6877185376310919
Fold: 3, logloss: 0.7742460414301828


[I 2023-03-30 22:11:15,467] Trial 0 finished with value: 0.8053286033369454 and parameters: {'n_estimators': 73, 'max_depth': 94}. Best is trial 0 with value: 0.8053286033369454.


Fold: 4, logloss: 0.771067862486872
Fold: 0, logloss: 0.6349545810814862
Fold: 1, logloss: 0.6842050323891357
Fold: 2, logloss: 0.6623077461302544


[I 2023-03-30 22:11:16,170] Trial 1 finished with value: 0.6810676824551527 and parameters: {'n_estimators': 65, 'max_depth': 16}. Best is trial 1 with value: 0.6810676824551527.


...

Fold: 4, logloss: 0.6891531393276241
Fold: 0, logloss: 0.6334503504438781
Fold: 1, logloss: 0.8853163769723198
Fold: 2, logloss: 0.7098915711714825
Fold: 3, logloss: 0.7581845467481555


[I 2023-03-30 22:11:36,063] Trial 28 finished with value: 0.7458809058033808 and parameters: {'n_estimators': 92, 'max_depth': 31}. Best is trial 2 with value: 0.6742602505132146.


Fold: 4, logloss: 0.7425616836810685
Fold: 0, logloss: 0.6255415590731161
Fold: 1, logloss: 1.2138698748837482
Fold: 2, logloss: 0.8529853015192349
Fold: 3, logloss: 0.7913816199047744


[I 2023-03-30 22:11:37,080] Trial 29 finished with value: 0.8495802148189886 and parameters: {'n_estimators': 74, 'max_depth': 82}. Best is trial 2 with value: 0.6742602505132146.


Fold: 4, logloss: 0.7641227187140694
best trial:
[0.6742602505132146]
Best parameters: {'n_estimators': 28, 'max_depth': 11}


## Train/validate/test
1. Hyperparameters were saved in config.py
2. Models were train/validate/test using 5-fold CV method and repeated for 5 times

In [35]:
def train_test_rfc_model(params, n_repetitions, X_train, X_test, y_train, y_test):
    rf_model = RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'])
    kf = KFold(n_splits=N_SPLITS)
    logloss_total = []
    acc_total  = []
    f1_total = []
    roc_auc_total = []
    for repeat in range(n_repetitions):
        for fold_no, (train_idx, valid_idx) in enumerate(kf.split(X_train, y_train)):
            seed_everything(SEED_NO)
            rf_model.fit(X_train.iloc[train_idx, :], y_train.iloc[train_idx].to_numpy().reshape(-1))
            y_pred = rf_model.predict(X_test)
            y_probs = rf_model.predict_proba(X_test)

            logloss= log_loss(y_test, y_probs)
            logloss_total.append(logloss)

            acc = accuracy_score(y_test, y_pred)
            acc_total.append(acc)

            f1 = f1_score(y_test,y_pred)
            f1_total.append(f1)

            roc_auc = roc_auc_score(y_test, y_probs[:,1])
            roc_auc_total.append(roc_auc)

    return mean(logloss_total), std(logloss_total), mean(acc_total), std(acc_total), mean(f1_total), std(f1_total), mean(roc_auc_total), std(roc_auc_total)

In [38]:
# For mol desc

df = pd.read_csv('./data/oral_avail_mol_desc/data_oral_avail_train_mol_desc.csv')
df_test = pd.read_csv('./data/oral_avail_mol_desc/data_oral_avail_test_mol_desc.csv')
x_train = df.iloc[:, :45]
y_train = df['label_cutoff_50%']

x_test = df_test.iloc[:, :45]
y_test = df_test['label_cutoff_50%']
params = best_params_MD
n_repetitions = 5

ll_mean, ll_std, acc_mean, acc_std, f1_mean, f1_std, roc_auc_mean, roc_auc_std = train_test_rfc_model(params,n_repetitions, x_train, x_test, y_train, y_test)
print(f'logloss: {ll_mean:.3f}±{ll_std:.3f}')
print(f'acc: {acc_mean:.3f}±{acc_std:.3f}')
print(f'f1: {f1_mean:.3f}±{f1_std:.3f}')
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_std:.3f}')


logloss: 0.556±0.008
acc: 0.724±0.012
f1: 0.769±0.011
roc_auc: 0.790±0.010


In [40]:
# For morgan

df = pd.read_csv('./data/oral_avail_fingerprints/data_oral_avail_train_morgan.csv')
df_test = pd.read_csv('./data/oral_avail_fingerprints/data_oral_avail_test_morgan.csv')
x_train = df.iloc[:, :1024]
y_train = df['label_cutoff_50%']

x_test = df_test.iloc[:, :1024]
y_test = df_test['label_cutoff_50%']
params = best_params_morgan
n_repetitions = 5

ll_mean, ll_std, acc_mean, acc_std, f1_mean, f1_std, roc_auc_mean, roc_auc_std = train_test_rfc_model(params,n_repetitions, x_train, x_test, y_train, y_test)
print(f'logloss: {ll_mean:.3f}±{ll_std:.3f}')
print(f'acc: {acc_mean:.3f}±{acc_std:.3f}')
print(f'f1: {f1_mean:.3f}±{f1_std:.3f}')
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_std:.3f}')


logloss: 0.592±0.015
acc: 0.688±0.021
f1: 0.738±0.027
roc_auc: 0.749±0.019


In [41]:
# For rdkfp

df = pd.read_csv('./data/oral_avail_fingerprints/data_oral_avail_train_rdkfp.csv')
df_test = pd.read_csv('./data/oral_avail_fingerprints/data_oral_avail_test_rdkfp.csv')
x_train = df.iloc[:, :1024]
y_train = df['label_cutoff_50%']

x_test = df_test.iloc[:, :1024]
y_test = df_test['label_cutoff_50%']
params = best_params_rdkfp
n_repetitions = 5

ll_mean, ll_std, acc_mean, acc_std, f1_mean, f1_std, roc_auc_mean, roc_auc_std = train_test_rfc_model(params,n_repetitions, x_train, x_test, y_train, y_test)
print(f'logloss: {ll_mean:.3f}±{ll_std:.3f}')
print(f'acc: {acc_mean:.3f}±{acc_std:.3f}')
print(f'f1: {f1_mean:.3f}±{f1_std:.3f}')
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_std:.3f}')


logloss: 0.631±0.019
acc: 0.665±0.009
f1: 0.724±0.011
roc_auc: 0.715±0.008


In [42]:
# For maccskeys

df = pd.read_csv('./data/oral_avail_fingerprints/data_oral_avail_train_maccskeys.csv')
df_test = pd.read_csv('./data/oral_avail_fingerprints/data_oral_avail_test_maccskeys.csv')
x_train = df.iloc[:, :167]
y_train = df['label_cutoff_50%']

x_test = df_test.iloc[:, :167]
y_test = df_test['label_cutoff_50%']
params = best_params_maccskeys
n_repetitions = 5

ll_mean, ll_std, acc_mean, acc_std, f1_mean, f1_std, roc_auc_mean, roc_auc_std = train_test_rfc_model(params,n_repetitions, x_train, x_test, y_train, y_test)
print(f'logloss: {ll_mean:.3f}±{ll_std:.3f}')
print(f'acc: {acc_mean:.3f}±{acc_std:.3f}')
print(f'f1: {f1_mean:.3f}±{f1_std:.3f}')
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_std:.3f}')


logloss: 0.584±0.004
acc: 0.688±0.018
f1: 0.741±0.016
roc_auc: 0.750±0.007
